In [1]:
import pandas as pd
import torch
import sys
sys.path.append('../')

from audio_slowfast.utils.metrics import multitask_topk_accuracies, topk_accuracies

In [2]:
annotations_path = "../data/epic-kitchens-100-annotations/EPIC_100_validation.pkl"
annotations = pd.read_pickle(annotations_path)
annotations.head()

,participant_id,video_id,narration_timestamp,start_timestamp,stop_timestamp,start_frame,stop_frame,narration,verb,verb_class,noun,noun_class,all_nouns,all_noun_classes
narration_id,,,,,,,,,,,,,,
P01_11_0,P01,P01_11,00:00:00.560,00:00:00.00,00:00:01.89,1,113,take plate,take,0,plate,2,[plate],[2]
P01_11_1,P01,P01_11,00:00:01.700,00:00:01.56,00:00:02.45,93,147,put down plate,put-down,1,plate,2,[plate],[2]
P01_11_10,P01,P01_11,00:00:48.500,00:00:49.15,00:00:50.95,2949,3057,take paper,take,0,paper,49,[paper],[49]
P01_11_100,P01,P01_11,00:05:27.840,00:05:27.28,00:05:31.97,19636,19918,wash cloth,wash,2,cloth,17,[cloth],[17]
P01_11_101,P01,P01_11,00:05:26.840,00:05:27.37,00:05:29.86,19642,19791,take cloth,take,0,cloth,17,[cloth],[17]


In [3]:
import numpy as np


a_path = "../audio-gru-cm.pkl"
v_path = "../sf-cm.pkl"

a_df = pd.read_pickle(a_path)
v_df = pd.read_pickle(v_path)

mm_df_sum = a_df.copy()
del mm_df_sum["verb_output"]
del mm_df_sum["noun_output"]

mm_df_sum["verb_output"] = v_df["verb_output"] + a_df["verb_output"]
mm_df_sum["noun_output"] = v_df["noun_output"] + a_df["noun_output"]

In [36]:
def compute_accuracy_at_k(df, labels, ks):
    for k in ks:
        verb_acc = topk_accuracies(
            torch.from_numpy(df["verb_output"]),
            torch.from_numpy(labels["verb_class"].values),
            (k,),
        )[0].item()
        noun_acc = topk_accuracies(
            torch.from_numpy(df["noun_output"]),
            torch.from_numpy(labels["noun_class"].values),
            (k,),
        )[0].item()

        action_acc = multitask_topk_accuracies(
            (torch.from_numpy(df["verb_output"]), torch.from_numpy(df["noun_output"])),
            (torch.from_numpy(labels["verb_class"].values), torch.from_numpy(labels["noun_class"].values)),
            (k,),
        )[0].item()

        print(f"\tAction Acc@{k}: {action_acc:.2f}")
        print(f"\tVerb Acc@{k}: {verb_acc:.2f}")
        print(f"\tNoun Acc@{k}: {noun_acc:.2f}")


print("AudioSlowFastGRU:")
compute_accuracy_at_k(a_df, annotations, (1, 5))

print("SlowFast:")
compute_accuracy_at_k(v_df, annotations, (1, 5))

print("MMLate-SlowFastGRU:")
compute_accuracy_at_k(mm_df_sum, annotations, (1, 5))

AudioSlowFastGRU:
	Action Acc@1: 17.06
	Verb Acc@1: 47.39
	Noun Acc@1: 24.24
	Action Acc@5: 44.30
	Verb Acc@5: 80.28
	Noun Acc@5: 49.49
SlowFast:
	Action Acc@1: 38.54
	Verb Acc@1: 65.56
	Noun Acc@1: 50.02
	Action Acc@5: 70.28
	Verb Acc@5: 90.00
	Noun Acc@5: 75.62
MMLate-SlowFastGRU:
	Action Acc@1: 39.04
	Verb Acc@1: 66.31
	Noun Acc@1: 50.37
	Action Acc@5: 70.45
	Verb Acc@5: 90.33
	Noun Acc@5: 75.70


In [5]:
annotations.verb_class.unique().shape

(78,)

In [6]:
1/78 * 100

1.282051282051282

In [6]:
from audio_slowfast.models.audio_model_builder import AudioSlowFast
from audio_slowfast.config.defaults import get_cfg


cfg_path = "../models/asf/config/asf-augment.yaml"
cfg = get_cfg()
cfg.merge_from_file(cfg_path)

cfg.NUM_GPUS = 0

model = AudioSlowFast(cfg)

In [4]:
# Print the number of parameters
num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters: {num_params:,}")

Number of parameters: 26,882,293


In [8]:
#!/usr/bin/env python3
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved.

import json
import logging
import math
import numpy as np
import os
from datetime import datetime
import psutil
import torch
from fvcore.common.file_io import PathManager
from fvcore.nn.activation_count import activation_count
from fvcore.nn.flop_count import flop_count
from matplotlib import pyplot as plt
from torch import nn

import audio_slowfast.utils.multiprocessing as mpu
from audio_slowfast.datasets.utils import pack_pathway_output
from audio_slowfast.models.batchnorm_helper import SubBatchNorm2d


def check_nan_losses(loss):
    """
    Determine whether the loss is NaN (not a number).
    Args:
        loss (loss): loss to check whether is NaN.
    """
    if math.isnan(loss):
        raise RuntimeError("ERROR: Got NaN losses {}".format(datetime.now()))


def params_count(model, ignore_bn=False):
    """
    Compute the number of parameters.
    Args:
        model (model): model to count the number of parameters.
    """
    if not ignore_bn:
        return np.sum([p.numel() for p in model.parameters()]).item()
    else:
        count = 0
        for m in model.modules():
            if not isinstance(m, nn.BatchNorm2d):
                for p in m.parameters(recurse=False):
                    count += p.numel()
    return count


def gpu_mem_usage():
    """
    Compute the GPU memory usage for the current device (GB).
    """
    if torch.cuda.is_available():
        mem_usage_bytes = torch.cuda.max_memory_allocated()
    else:
        mem_usage_bytes = 0
    return mem_usage_bytes / 1024**3


def cpu_mem_usage():
    """
    Compute the system memory (RAM) usage for the current device (GB).
    Returns:
        usage (float): used memory (GB).
        total (float): total memory (GB).
    """
    vram = psutil.virtual_memory()
    usage = (vram.total - vram.available) / 1024**3
    total = vram.total / 1024**3

    return usage, total


def _get_model_analysis_input(cfg):
    """
    Return a dummy input for model analysis with batch size 1. The input is
        used for analyzing the model (counting flops and activations etc.).
    Args:
        cfg (CfgNode): configs. Details can be found in
            slowfast/config/defaults.py

    Returns:
        inputs: the input for model analysis.
    """
    spectrogram_dimension = 1
    input_tensors = torch.rand(
        spectrogram_dimension,
        cfg.AUDIO_DATA.NUM_FRAMES,
        cfg.AUDIO_DATA.NUM_FREQUENCIES,
    )
    model_inputs = pack_pathway_output(cfg, input_tensors)
    for i in range(len(model_inputs)):
        model_inputs[i] = model_inputs[i].unsqueeze(0)
        if cfg.NUM_GPUS:
            model_inputs[i] = model_inputs[i].cuda(non_blocking=True)

    inputs = (model_inputs,)
    return inputs


def get_model_stats(model, cfg, mode):
    """
    Compute statistics for the current model given the config.
    Args:
        model (model): model to perform analysis.
        cfg (CfgNode): configs. Details can be found in
            slowfast/config/defaults.py
        mode (str): Options include `flop` or `activation`. Compute either flop
            (gflops) or activation count (mega).

    Returns:
        float: the total number of count of the given model.
    """
    assert mode in [
        "flop",
        "activation",
    ], "'{}' not supported for model analysis".format(mode)
    if mode == "flop":
        model_stats_fun = flop_count
    elif mode == "activation":
        model_stats_fun = activation_count

    # Set model to evaluation mode for analysis.
    # Evaluation mode can avoid getting stuck with sync batchnorm.
    model_mode = model.training
    model.eval()
    inputs = _get_model_analysis_input(cfg)
    count_dict, *_ = model_stats_fun(model, inputs)
    count = sum(count_dict.values())
    model.train(model_mode)
    return count


def log_model_info(model, cfg):
    """
    Log info, includes number of parameters, gpu usage, gflops and activation count.
        The model info is computed when the model is in validation mode.
    Args:
        model (model): model to log the info.
        cfg (CfgNode): configs. Details can be found in
            slowfast/config/defaults.py
    """
    logger.info("Model:\n{}".format(model))
    logger.info("Params: {:,}".format(params_count(model)))
    logger.info("Mem: {:,} MB".format(gpu_mem_usage()))
    logger.info("Flops: {:,} G".format(get_model_stats(model, cfg, "flop")))
    logger.info("Activations: {:,} M".format(get_model_stats(model, cfg, "activation")))
    logger.info("nvidia-smi")
    os.system("nvidia-smi")


def is_eval_epoch(cfg, cur_epoch):
    """
    Determine if the model should be evaluated at the current epoch.
    Args:
        cfg (CfgNode): configs. Details can be found in
            slowfast/config/defaults.py
        cur_epoch (int): current epoch.
    """
    if cur_epoch + 1 == cfg.SOLVER.MAX_EPOCH:
        return True

    return (cur_epoch + 1) % cfg.TRAIN.EVAL_PERIOD == 0


def plot_input(tensor, bboxes=(), texts=(), path="./tmp_vis.png"):
    """
    Plot the input tensor with the optional bounding box and save it to disk.
    Args:
        tensor (tensor): a tensor with shape of `NxCxHxW`.
        bboxes (tuple): bounding boxes with format of [[x, y, h, w]].
        texts (tuple): a tuple of string to plot.
        path (str): path to the image to save to.
    """
    tensor = tensor.float()
    tensor = tensor - tensor.min()
    tensor = tensor / tensor.max()
    f, ax = plt.subplots(nrows=1, ncols=tensor.shape[0], figsize=(50, 20))
    for i in range(tensor.shape[0]):
        ax[i].axis("off")
        ax[i].imshow(tensor[i].permute(1, 2, 0))
        # ax[1][0].axis('off')
        if bboxes is not None and len(bboxes) > i:
            for box in bboxes[i]:
                x1, y1, x2, y2 = box
                ax[i].vlines(x1, y1, y2, colors="g", linestyles="solid")
                ax[i].vlines(x2, y1, y2, colors="g", linestyles="solid")
                ax[i].hlines(y1, x1, x2, colors="g", linestyles="solid")
                ax[i].hlines(y2, x1, x2, colors="g", linestyles="solid")

        if texts is not None and len(texts) > i:
            ax[i].text(0, 0, texts[i])
    f.savefig(path)


def aggregate_sub_bn_stats(module):
    """
    Recursively find all SubBN modules and aggregate sub-BN stats.
    Args:
        module (nn.Module)
    Returns:
        count (int): number of SubBN module found.
    """
    count = 0
    for child in module.children():
        if isinstance(child, SubBatchNorm2d):
            child.aggregate_stats()
            count += 1
        else:
            count += aggregate_sub_bn_stats(child)
    return count


def launch_job(cfg, init_method, func, daemon=False):
    """
    Run 'func' on one or more GPUs, specified in cfg
    Args:
        cfg (CfgNode): configs. Details can be found in
            slowfast/config/defaults.py
        init_method (str): initialization method to launch the job with multiple
            devices.
        func (function): job to run on GPU(s)
        daemon (bool): The spawned processes’ daemon flag. If set to True,
            daemonic processes will be created
    """
    if cfg.NUM_GPUS > 1:
        torch.multiprocessing.spawn(
            mpu.run,
            nprocs=cfg.NUM_GPUS,
            args=(
                cfg.NUM_GPUS,
                func,
                init_method,
                cfg.SHARD_ID,
                cfg.NUM_SHARDS,
                cfg.DIST_BACKEND,
                cfg,
            ),
            daemon=daemon,
        )
    else:
        func(cfg=cfg)


def get_class_names(path, parent_path=None, subset_path=None):
    """
    Read json file with entries {classname: index} and return
    an array of class names in order.
    If parent_path is provided, load and map all children to their ids.
    Args:
        path (str): path to class ids json file.
            File must be in the format {"class1": id1, "class2": id2, ...}
        parent_path (Optional[str]): path to parent-child json file.
            File must be in the format {"parent1": ["child1", "child2", ...], ...}
        subset_path (Optional[str]): path to text file containing a subset
            of class names, separated by newline characters.
    Returns:
        class_names (list of strs): list of class names.
        class_parents (dict): a dictionary where key is the name of the parent class
            and value is a list of ids of the children classes.
        subset_ids (list of ints): list of ids of the classes provided in the
            subset file.
    """
    try:
        with PathManager.open(path, "r") as f:
            class2idx = json.load(f)
    except Exception as err:
        print("Fail to load file from {} with error {}".format(path, err))
        return

    max_key = max(class2idx.values())
    class_names = [None] * (max_key + 1)

    for k, i in class2idx.items():
        class_names[i] = k

    class_parent = None
    if parent_path is not None and parent_path != "":
        try:
            with PathManager.open(parent_path, "r") as f:
                d_parent = json.load(f)
        except EnvironmentError as err:
            print("Fail to load file from {} with error {}".format(parent_path, err))
            return
        class_parent = {}
        for parent, children in d_parent.items():
            indices = [class2idx[c] for c in children if class2idx.get(c) is not None]
            class_parent[parent] = indices

    subset_ids = None
    if subset_path is not None and subset_path != "":
        try:
            with PathManager.open(subset_path, "r") as f:
                subset = f.read().split("\n")
                subset_ids = [class2idx[name] for name in subset if class2idx.get(name) is not None]
        except EnvironmentError as err:
            print("Fail to load file from {} with error {}".format(subset_path, err))
            return

    return class_names, class_parent, subset_ids

In [9]:
get_model_stats(model, cfg, "flop")

4.332613887999999